<a href="https://colab.research.google.com/github/kvinne-anc/SpotifyApp-Project/blob/master/Spotify_Plotly_DashApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install jupyter-dash

     |████████████████████████████████| 81kB 3.4MB/s 
     |████████████████████████████████| 1.0MB 6.9MB/s 
     |████████████████████████████████| 3.5MB 12.4MB/s 
     |████████████████████████████████| 194kB 29.5MB/s 
     |████████████████████████████████| 1.8MB 34.8MB/s 
     |████████████████████████████████| 358kB 35.1MB/s 
  Created wheel for dash: filename=dash-1.19.0-cp37-none-any.whl size=84011 sha256=4afde6c307caf353d7e77a26f5dafbf50343e3548418ccdabfbfd169de299076
  Stored in directory: /root/.cache/pip/wheels/f7/50/a7/a230ff7f503b10120bff18c2524a375bb85a61ce6b519c8a77
  Created wheel for flask-compress: filename=Flask_Compress-1.9.0-cp37-none-any.whl size=7259 sha256=eb2f459247c50b9ccee4eacb849535e7b2df542742f30472ad9d19fb172a0b61
  Stored in directory: /root/.cache/pip/wheels/0d/56/7d/e0c50541e5421b8291d958afb098c23818989e29d91a1edac5
  Created wheel for dash-renderer: filename=dash_renderer-1.9.0-cp37-none-any.whl size=1014870 sha256=68654c4be4ad2e493b2d97ea38504d970ebf0

In [ ]:
import numpy as np
import pandas as pd 
from scipy import stats
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
import dash

#import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [ ]:
#def configure_plotly_browser_state():
 # import IPython
  #display(IPython.core.display.HTML('''
  #<script src="/static/components/requirejs/require.js"></script>
  #<script>
  #requirejs.config({
   # paths: {
    #  base: '/static/base',
     # plotly: 'http://cdn.plot.ly/plotly-1.5.1.min.js?noext', 
   # },
  #});
  #</script>
  #'''))

In [ ]:
dataset = pd.read_csv("/content/data.csv")
dataset

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.98200,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.6650,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.73200,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.1600,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.96100,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.1010,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.96700,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.3810,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.95700,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.2290,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170648,0.6080,2020,0.08460,"['Anuel AA', 'Daddy Yankee', 'KAROL G', 'Ozuna...",0.786,301714,0.808,0,0KkIkfsLEJbrcIhYsCL7L5,0.000289,7,0.0822,-3.702,1,China,72,2020-05-29,0.0881,105.029
170649,0.7340,2020,0.20600,['Ashnikko'],0.717,150654,0.753,0,0OStKKAuXlxA0fMH54Qs6E,0.000000,7,0.1010,-6.020,1,Halloweenie III: Seven Days,68,2020-10-23,0.0605,137.936
170650,0.6370,2020,0.10100,['MAMAMOO'],0.634,211280,0.858,0,4BZXVFYCb76Q0Klojq4piV,0.000009,4,0.2580,-2.226,0,AYA,76,2020-11-03,0.0809,91.688
170651,0.1950,2020,0.00998,['Eminem'],0.671,337147,0.623,1,5SiZJoLXp3WOl3J4C8IK0d,0.000008,2,0.6430,-7.161,1,Darkness,70,2020-01-17,0.3080,75.055


In [ ]:
#Will probably need to make a new DF from the results of the model, fetching the elements we want to map

In [ ]:
instrumentalness = ["0.000001", "0.000005", "0.000010", "0.000050", "0.000100", "0.000500", "0.001000", "0.005000", "0.010000", "0.050000", "0.100000", "0.500000",
         "1.000000"]
#Here we want to check the range of the popularity element and map it in increments in place of years. 
#Need to also make sure the columns we are using are formatted uniformily - ie. years seems to be in multiple formats but it might be the only one - clean up the dataset

#sub artists - this is a string and there will only be ten, they will be in the legend on the side
#figure out if we need to differentiate by using singular and plural forms (artist in artists) - it is put this way now:
#year = popularity, years = popularities, continent = artists, continents = artist

names = []
for name in dataset["name"]:
    if name not in names:
        names.append(name)

In [ ]:
# make figure
fig_dict = {
    "data": [],
    "layout": {},
    "frames": []
}

In [ ]:
# fill in most of layout
fig_dict["layout"]["xaxis"] = {"range": [100000, 900000], "title": "Duration"}
fig_dict["layout"]["yaxis"] = {"title": "Tempo", "type": "log"}
fig_dict["layout"]["hovermode"] = "closest"
fig_dict["layout"]["updatemenus"] = [
    {
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 500, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 300,
                                                                    "easing": "quadratic-in-out"}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }
]

sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Popularity:",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}

In [ ]:
# make data - switch instrumentalness and popularity
#switch name and artists
#sizemode may need to be changed from area if that is referring to actual land area, but that isn't a column in OG set so prob not?


instrumentalness = 0.500000
for name in names:
    dataset_by_pop = dataset[dataset["instrumentalness"] == instrumentalness]
    #dataset_by_pop_and_art = dataset_by_pop[dataset_by_pop["name"] == name]

    data_dict = {
        "x": list(dataset_by_pop["duration_ms"]),
        "y": list(dataset_by_pop["tempo"]),
        "mode": "markers",
        "text": list(dataset_by_pop["name"]),
        "marker": {
            "sizemode": "area",
            "sizeref": 200000,
            "size": list(dataset_by_pop["popularity"])
        },
        "name": name
    }
    fig_dict["data"].append(data_dict)

In [ ]:
#dataset_by_pop_and_art = dataset_by_pop[dataset_by_pop["name"] == name]

In [ ]:
dataset_by_pop

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
17285,0.1120,2008,0.054200,['Kanye West'],0.760,270307,0.5240,0,1kxeWHF9PrCVZHvVskv8lg,0.5,1,0.1120,-7.670,0,Love Lockdown,67,2008-11-24,0.0323,119.603
22623,0.4500,1939,0.993000,['Jack Teagarden And His Orchestra'],0.573,112387,0.1230,0,0Mp5dTBbS6nv2DEC7mnjGg,0.5,8,0.1350,-15.446,1,"Theme and Opening - Aug 23, 1939",0,1939,0.1250,81.509
23004,0.9610,1941,0.979000,['Carmen Miranda'],0.780,150307,0.4260,0,1jKTNd8P12QM8tiUGsAvL8,0.5,1,0.0698,-12.594,0,Uva De Caminhão,1,1941,0.2150,103.307
23824,0.0395,1945,0.960000,"['George Frideric Handel', 'Yehudi Menuhin']",0.148,245507,0.1640,0,09h3g1cXbKmbaO2AzdE9U4,0.5,6,0.1600,-18.638,0,"Te Deum in D, HWV 283 ""Dettingen"": Prayer",0,1945,0.0415,84.996
25342,0.5340,1953,0.995000,['Cisco Houston'],0.507,107187,0.0338,0,7zECKmOUwx7V59i3fAuHIi,0.5,9,0.1080,-26.675,1,Railroad Bill,5,1953-01-01,0.2010,76.677
27898,0.5510,1966,0.865000,['Thad Jones-Mel Lewis Jazz Orchestra'],0.366,348733,0.2570,0,5NdEAODwKFtl1ZJ0Bjyk9h,0.5,8,0.1040,-21.669,1,Three In One,36,1966-02-16,0.0875,171.876
30630,0.6130,1980,0.434000,['Dead Kennedys'],0.219,133040,0.9760,0,0WFI6O0j9BDwrHtZrScbj5,0.5,11,0.3090,-2.934,1,Let's Lynch the Landlord,39,1980-09-02,0.0967,161.285
39493,0.1910,1930,0.991000,['Z.Kwartin'],0.261,233013,0.0889,0,1mv5ayzYScVNmSc1ut81q2,0.5,10,0.1260,-17.633,0,L'dovid mizmor,0,1930,0.0347,140.977
41529,0.8380,1941,0.995000,['Leela Chitnis'],0.642,190990,0.2260,0,3rNIKoNMRfyEJWLFk8x4Ae,0.5,1,0.2690,-11.238,1,Mori Gali Aa Re Sajan,0,1941-12-31,0.4310,118.934
42796,0.8060,1947,0.996000,['Rajkumari'],0.649,185921,0.1520,0,22OGjfKEl4ORKqjsqhUyCv,0.5,7,0.2300,-10.145,1,Piyu Paranyani Raatadi Pehli,0,1947-03-27,0.3060,143.113


In [ ]:
data_dict

{'marker': {'size': [67,
   0,
   1,
   0,
   5,
   36,
   39,
   0,
   0,
   0,
   0,
   26,
   36,
   56,
   0,
   17,
   31,
   55,
   47,
   6,
   42],
  'sizemode': 'area',
  'sizeref': 200000},
 'mode': 'markers',
 'name': 'Billetes Azules (with J Balvin)',
 'text': ['Love Lockdown',
  'Theme and Opening - Aug 23, 1939',
  'Uva De Caminhão',
  'Te Deum in D, HWV 283 "Dettingen": Prayer',
  'Railroad Bill',
  'Three In One',
  "Let's Lynch the Landlord",
  "L'dovid mizmor",
  'Mori Gali Aa Re Sajan',
  'Piyu Paranyani Raatadi Pehli',
  'Gaiya Dana Bhusa Khaye',
  'Milk Cow Blues',
  'Down In It',
  'We Can Make The World Stop',
  "I'd Rather Cry Over You",
  'Theme from "A Summer Place"',
  'Love Theme',
  "Block Rockin' Beats",
  'The Cedar Room',
  'Pretend',
  "Don't Let It Pass"],
 'x': [270307,
  112387,
  150307,
  245507,
  107187,
  348733,
  133040,
  233013,
  190990,
  185921,
  184947,
  220893,
  226467,
  325909,
  161587,
  142920,
  268200,
  294027,
  458933,
  16

In [ ]:
#configure_plotly_browser_state()
#init_notebook_mode(connected=False)
# make frames
for instrumentalness in instrumentalness:
    frame = {"data": [], "name": ("instrumentalness")}
    for name in names:
        dataset_by_pop = dataset[dataset["instrumentalness"] == ("instrumentalness")]
        #dataset_by_pop_and_art = dataset_by_pop[
           # dataset_by_pop["name"] == name]

        data_dict = {
            "x": list(dataset_by_pop["duration_ms"]),
            "y": list(dataset_by_pop["tempo"]),
            "mode": "markers",
            "text": list(dataset_by_pop["name"]),
            "marker": {
                "sizemode": "area",
                "sizeref": 200000,
                "size": list(dataset_by_pop_and_art["speechiness"])
            },
            "name": name
        }
        frame["data"].append(data_dict)

    fig_dict["frames"].append(frame)
    slider_step = {"args": [
        [popularity],
        {"frame": {"duration": 300, "redraw": False},
         "mode": "immediate",
         "transition": {"duration": 300}}
    ],
        "label": popularity,
        "method": "animate"}
    sliders_dict["steps"].append(slider_step)


fig_dict["layout"]["sliders"] = [sliders_dict]

fig = go.Figure(fig_dict)

fig.show()
#iplot({"data":data,
 #      "layout":layout})

# iplot({fig})

TypeError: ignored